In [9]:
# return to parent directory
%cd ..

F:\Code\Finance


In [10]:
# load modulers
from engine.allocation import *
from engine.utils import *
import time
from scipy.stats import f_oneway

In [11]:
# set up parameters
date_skip = 1
before_dates = 30
after_dates = 60
before_idx = before_dates // date_skip
after_idx = after_dates // date_skip
dates = get_trade_dates(['2000-01-01', '2022-01-01'])
dates = [dates[i] for i in range(0, len(dates), date_skip)]

In [12]:
# load indicators

# load turnover
TO_path = os.path.join(ARR_DATA, 'Ind', 'Daily', 'Turnover.csv')
TO = pd.read_csv(TO_path, index_col=0)

# load adjusted turnover
adjTO_path = os.path.join(ARR_DATA, 'Ind', 'Daily', 'AdjTO.csv')
adjTO = pd.read_csv(adjTO_path, index_col=0)

# load market value
MV_path = os.path.join(ARR_DATA, 'Ret', 'Daily', 'Dsmvtll.csv')
MV = pd.read_csv(MV_path, index_col='Trddt')

In [13]:
# allocate stocks
feature_dfs = [MV, adjTO]
periods = [(dates[idx - before_idx], dates[idx + after_idx]) for idx in
           range(before_idx, len(dates[before_idx:-after_idx]))]
features = [[feature.loc[date] for feature in feature_dfs] for date in dates[before_idx:-after_idx]]

allocation = allocate_all_periods(periods, features, [(0, 0.5, 1), (0, 0.3, 0.7, 1)], ['MV', 'adjTO'])
print(allocation)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
mkt_val_all = get_mkt_val(0)
ret_val_all = get_ret_val(1)

all_p = {}
for name, p in allocation.items():
    t = time.time()
    all_p[name] = []
    for date, portfolios in p:
        all_p[name].append(weighted_return(mkt_val_all, ret_val_all, date, portfolios).reset_index(drop=True))
    all_p[name] = pd.DataFrame(all_p[name])
print(all_p)

{'MV0adjTO0':             0         1         2         3         4         5         6   \
0     0.023875  0.005646  0.027571  0.032335  0.018036 -0.042992 -0.022799   
1     0.008659  0.032922  0.033902  0.021024 -0.038041 -0.023598 -0.011990   
2     0.031698  0.034568  0.025565 -0.043250 -0.025953 -0.010103 -0.014027   
3     0.035945  0.021035 -0.040319 -0.021481 -0.009916 -0.011098  0.013429   
4     0.019503 -0.037726 -0.021974 -0.007446 -0.011206  0.014733 -0.000377   
...        ...       ...       ...       ...       ...       ...       ...   
5182  0.007642  0.004546 -0.005590  0.009563  0.009830 -0.006707 -0.001378   
5183  0.004070 -0.005328  0.011303  0.011249 -0.007157 -0.000888 -0.003522   
5184 -0.004865  0.009072  0.010127 -0.006795 -0.001910 -0.005181  0.004765   
5185  0.009865  0.009633 -0.006769  0.000110 -0.003543  0.005601  0.001371   
5186  0.009977 -0.005403  0.000262 -0.004098  0.006788  0.001829  0.005753   

            7         8         9   ...        80